<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/BERTSum_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/nlpyang/BertSum

https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1214/reports/final_reports/report042.pdf

https://github.com/sudharsan13296/Getting-Started-with-Google-BERT/blob/main/6.%20Exploring%20BERTSUM%20for%20text%20summarization/6.07.%20Training%20the%20BERTSUM%20model%20.ipynb

In [ ]:
# Step 1: Install dependencies
!pip install transformers
!pip install torch

In [13]:
# Step 2: Import necessary libraries
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import softmax
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
file_path = './saved_file'
file_path = ('DataSampePilot.xlsx')

data = pd.read_excel(file_path)

In [ ]:
# Display the first few rows of the dataset
data.head()

# Clone the Repository and Navigate to It
https://chatgpt.com/share/d9a17184-b011-4f11-85e4-7ab2636bbd30

In [ ]:
!git clone https://github.com/nlpyang/BertSum.git


In [14]:
# List the files in the current directory
!ls

BertSum  DataSampePilot.xlsx  sample_data


In [13]:
!ls BertSum

bert_config_uncased_base.json  json_data  logs	  raw_data   results  urls
bert_data		       LICENSE	  models  README.md  src


In [16]:
%cd BertSum

[Errno 2] No such file or directory: 'BertSum'
/content/BertSum


In [21]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
# Step 3: Define the BertSum model for extractive summarization
class BertSum:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def summarize(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
        outputs = self.model(**inputs)
        # Obtain the embeddings for each token
        token_embeddings = outputs.last_hidden_state
        # Average the token embeddings to get a sentence embedding
        sentence_embedding = torch.mean(token_embeddings, dim=1)
        # Compute softmax scores
        scores = softmax(sentence_embedding, dim=1)
        # Split the text into sentences
        sentences = sent_tokenize(text)
        # Get the top N sentences (e.g., top 3)
        N = min(3, len(sentences))  # Ensure N does not exceed the number of sentences
        top_sentence_idxs = scores[0].topk(N).indices.tolist()
        # Select the top N sentences, ensuring valid indices
        top_sentence_idxs = [idx for idx in top_sentence_idxs if idx < len(sentences)]
        summary = '. '.join([sentences[idx] for idx in top_sentence_idxs])
        return summary

In [23]:
# Initialize the model
bertsum = BertSum()

In [24]:
# Step 4: Apply the model to the Question_body and Answer_body columns
data['Question_summary'] = data['Question_body'].apply(lambda x: bertsum.summarize(x) if pd.notnull(x) else "")
data['Answer_summary'] = data['Answer_body'].apply(lambda x: bertsum.summarize(x) if pd.notnull(x) else "")


In [25]:
# Display the summarized data
data[['Question_body', 'Question_summary', 'Answer_body', 'Answer_summary']].head()


,Question_body,Question_summary,Answer_body,Answer_summary
0,Kinda new to AWS. I have this high-level quest...,,"You send a request, you get a response. In ord...",
1,I have some spring boot microservices and I wa...,,<blockquote>\ntl;dr: Spring MVC will not contr...,
2,I'm trying to properly design an application a...,,Determining the source of the information is b...,"Your repositories might return the same model,..."
3,I heard that for .NET8 Microsoft gifted us wit...,,I have always asked myself this very same ques...,
4,"I am trying to learn AWS services, and now it ...",,"Short answer is: no, you don't have to but you...",


In [26]:
# Save the summarized data to a new Excel file
data.to_excel('SummarizedData.xlsx', index=False)